<h1><center>Storm Surge Detailing Set Up</center></h1>

<h4><center>Jinpai (Max) Zhao</center></h4>

<hr style="border:0.5px solid gray">
<hr style="border:0.5px solid gray">

## Step 0: Validation Study

#### What is validation study?
    
Generally speaking, validation study is to compare the result by a measure or method to which by a standard or accurate model. In our case, we are comparing the GeoClaw simulated storm surge result to actual observed meteorological statistics.

#### Why is validation study important?
        
As a powerful software, GeoClaw aims to reconstruct natural disasters as precise as possible. With more examples from validation study, we can better identify and analyze discrepancy between simulated and observed data. Therefore, improvements can be made upon GeoClaw software. On the other hand, for unpredictable or unquantifiable problems, we are able to conclude which kinds of storms will be well-simulated by GeoClaw and which won't. 

<hr style="border:1.5px solid gray">

## Step 1: Setup Files and Directory

To get started with a storm surge computation it is best to refer to a previous working example. For example, if you have successfully installed Clawpack, you might start with `$ CLAW/geoclaw/examples/storm-surge/ike`. If you have not yet set up environment variable `CLAW`, please refer to this page: [Setting Environment Variable](http://www.clawpack.org/setenv.html). There are also a number of additional examples in the `$ CLAW/geoclaw/examples/storm-surge` directory as well as some in the `$ CLAW/apps/surge-examples directory`. 

The primary input that one needs to provide for a new example usually involves two data source: 

- Topography data: Data that specifies the topography and bathymetry of the region around the area of interest. For storm surge computations, it is generally good practice to include entire oceanic basins so that you can ensure that flow into and out of the basin is resolved by the computation and is sufficiently distant from the computational domain’s boundaries.

- Storm data: We also need to specify the particular storm that you are interested in. Storm specification data described and parametrized a storm which will be read in by GeoClaw. 

In this tutorial, we will concentrate on changing the Hurricane Ike (2018) example into one for Hurricane Elsa (2021).

First copy the files (setrun.py, setplot.py, makefile) located in the Hurricane Ike directorty located at `$ CLAW/geoclaw/examples/storm-surge/ike`. Then make a folder at your desired location with those three files included. Here I named this folder `elsa`.
<div>
<img src="pic1.png" width="800"/>
</div>

<hr style="border:1.5px solid gray">

## Step 2: Find Topography/Bathymetry Data

Before changing anything, we must know some information of the hurricane we are studying. One of the best way to do this is to go through the hurricane report written by the National Hurricane Center. For hurricane Elsa, this file locates [here](https://www.nhc.noaa.gov/data/tcr/AL052021_Elsa.pdf). In this report, we know the path of Elsa, strength along the path, specific landfall time, reading at different gauges, how it's affecting coastal communities, and so on. 

With the path of Elsa in our head, let’s find some better topography for Elsa which is the west coast Florida region. There are several places where we can get topography/bathymetry data, for example [Global Multi-Resolution Topography Data Synthesis](https://www.gmrt.org/index.php), [Global Digital Elevation Model](https://www.jspacesystems.or.jp/ersdac/GDEM/E/1.html), [National Center for Environmental Information](https://www.ngdc.noaa.gov/mgg/bathymetry/relief.html), and [The General Bathymetry Chart of the Oceans](https://www.gebco.net/data_and_products/gridded_bathymetry_data/). Here we will use the data from the last one which is the GEBCO to retrive and download the topography and bathymetry data for hurricane Elsa. 
<div>
<img src="pic2.png" width="800"/>
</div>

Theoretically, we can select the region as big as possible. However, the bigger the region, the more computationally complex it will be for GeoClaw simulation. So for hurricane Elsa, we only use the gulf of Mexico region so that it can include the region days before Elsa's landfall and the region days after the landfall. (How to specify how many days before and after will be discussed in later steps below.) After downloading the Esri ASCII + Grid format of the topography data, we may want to store it on cloud like google drive, dropbox, or your personal website so that people running your example in the future can have access to this topography data file.

<hr style="border:1.5px solid gray">

## Step 3: Find Storm Specific Data

Now let’s find a storm specification for Hurricane Elsa. Several data resouces are available: [Automated Tropical Cyclone Forecasting System](https://ftp.nhc.noaa.gov/atcf/archive/), [Atlantic Oceanographic and Meteorological Laboratory](https://www.aoml.noaa.gov/hrd/hurdat/Data_Storm.html), [NOAA Rapid Update Cycle](https://ruc.noaa.gov/tracks/), [National Center for Environmental Information](http://www.ncdc.noaa.gov/ibtracs/), [Naval Meteorology and Oceanography Command](https://www.metoc.navy.mil/jtwc/jtwc.html?best-tracks), [Coastal Hazards System](https://chs.erdc.dren.mil). In this example we will use the ATCF database. For Hurricane Elsa this ends up being the file located [here](http://ftp.nhc.noaa.gov/atcf/archive/2021/bal052021.dat.gz).

How to find a storm specific data using ATCF database for your storm? Go to ATCF Archive. 
<div>
<img src="pic3.png" width="300"/>
</div>

Find and go to the archive the year when you storm happened
<div>
<img src="pic4.png" width="300"/>
</div>

Then search for you storm number. In this example, storm number for Hurricane Elsa is AL052021. 
<div>
<img src="pic5.png" width="300"/>
</div>

Therefore, the storm data corresponds to Hurricane Elsa will simply be bal052021.dat in year 2021.

<hr style="border:1.5px solid gray">

## Step 4: Modify `setrun.py`

Now we have our topography data and storm specific data ready. Next, we need to modify the `setrun.py` to use our new storm specific data and topography data we just added. First, modify the computational domain so that it matches or contains in the region of the topography data:

```
# Lower and upper edge of computational domain for hurricane elsa:
    clawdata.lower[0] = -95.0      # longitude: west limit
    clawdata.upper[0] = -70.0      # longitude: east limit

    clawdata.lower[1] = 12.0       # latitude: south limit
    clawdata.upper[1] = 32.0       # latitude: north limit
```

Then change `t0`, `tfinal`, and `time_offset` to Elsa's landfall time and time period before and after landfall we want to simulate:

```
...
# -------------
# Initial time:
# -------------
clawdata.t0 = -days2seconds(2)
...
# -------------
# End time:
# -------------
clawdata.tfinal = days2seconds(1)
...
# -------------
# Landfall time:
# -------------
elsa.time_offset = datetime.datetime(2021, 7, 7, 14)
...

```

Last but not least, change locations of retrieving topography and storm data file in topography data and surge data sections:

```
...
# Fetch topography data
clawutil.data.get_remote_file(
           "ULR_for_topography_data")
...
# Fetch storm specification data
clawutil.data.get_remote_file(
           "URL_for_storm_specification_data")
...
```

<hr style="border:1.5px solid gray">

## Step 5: Modify `setplot.py`

We also need to modify the plotting in `setplot.py`. First, we may want to change upper and lower limits for surface levels, wind speeds, pressure, and friction to best represent our storm. For example, Elsa is a category 1 hurricane which is not very strong. Therefore, we changed the surface levels to `[-1, 1]` so that visualization from plots will be more evident. 

```
# User can modify these to best visualization for your storm
surface_limits = [-1.0, 1.0]
speed_limits = [0.0, 3.0]
wind_limits = [0, 64]
pressure_limits = [935, 1013]
friction_bounds = [0.01, 0.04]
```

In the plot specifications section, we can add zoom-in regions. For Elsa, three more regions other than the computational domain are added. The reason for adding more zoom in regions could be:
1. To better visualize what will happen in the zoom-in area when the storm passes by, verifying if it's similar to the description in hurricane report.
2. To see what will happen around gauges we set, testing if the behavior of gauge plots are consistent with GeoClaw simulation. 

We may also alter the legends, labels, colors of plots, and so on in `setplot.py`. 

```
# ==========================================================================
#   Plot specifications
# ==========================================================================
regions = {"Gulf": {"xlimits": (clawdata.lower[0], clawdata.upper[0]),
                    "ylimits": (clawdata.lower[1], clawdata.upper[1]),
                    "figsize": (6.4, 4.8)},
           "Cedar Key": {"xlimits": (-84, -82.5),
                           "ylimits": (28, 30.5),
                           "figsize": (2.5, 4)},
            "Apalachee Bay": {"xlimits": (-84.47, -82.38),
                           "ylimits": (29.4,30.17),
                           "figsize": (4.5, 2.5)},
            "Tampa Bay": {"xlimits": (-82.89, -82.38),
                           "ylimits": (27.49, 28.04),
                           "figsize": (3, 3.5)}}
```

For the figure sizes, you might want to approximate the ratio between the length and width of the zoom-in regions selected. Inappropriate ratios may result in bad visualizations and hence unrecognizable regions.  

<hr style="border:1.5px solid gray">

## Step 6: Add Gauges for Validation

Gauges are key for validation study. We want to know how good GeoClaw's simulations are comparing to observed currents and tides data. 

Gauge information and location can be found here at [NOAA Tides & Currents](https://tidesandcurrents.noaa.gov).  
<div>
<img src="pic6.png" width="800"/>
</div>

Choose the gauges that are along the path of the hurricane on the map. For example, here I choose to add gauge at Clearwater Beach, west Florida for hurricane Elsa.

<div>
<img src="pic7.png" width="800"/>
</div>

At the station's home page, we can see all kinds of data for this gauge. One thing really important is the station ID, because we will use this in `setplot.py` to fetch observed surge data. 

<div>
<img src="pic8.png" width="800"/>
</div>

After finalizing all the gauges you want to add, it's time to modify `setrun.py` and `setplot.py` accordingly, so that the program will plot the observed data along with the simulation data for those gauges you have selected. 

In `setrun.py`, change the gauges to your selection. Here for example, `1` means this is the first gauge added, and `-82.93, 27.98` is the exact location of the gauge at Clearwater Beach. 

```
rundata.gaugedata.gauges.append([1, -82.93, 27.98,
                                 rundata.clawdata.t0,
                                 rundata.clawdata.tfinal])   # CWBF1 Clearwater Beach, ID: 8726724
... (more gauges)
```

In `setplot.py`, modify the dictionary containing stations IDs and `gauge_afteraxes(cd)` method:

```
my_dict = {1: '8726724', .... add_more_gauegs}
def gauge_afteraxes(cd): 
    t0 = datetime.datetime(2021, 7, 5, 14)         # simulation start time
    t_offset = datetime.datetime(2021, 7, 7, 14)   # landfall
    tf = datetime.datetime(2021, 7, 8, 14)         # simulation end time
```

Also in `setplot.py`, change the region for the gauge location plot so that we can visualize where our gauges are on the map.

```
plotaxes = plotfigure.new_plotaxes()
plotaxes.title = 'Gauge Locations'
plotaxes.scaled = True
plotaxes.xlimits = [-90, -80]       # West and East limits
plotaxes.ylimits = [23.0, 31.0]     # South and North limits

```

<div>
<img src="pic9.png" width="500"/>
</div>

<hr style="border:1.5px solid gray">

## Step 7: Setup [AMRClaw Algorithm](https://www.clawpack.org/amrclaw.html)

Sometimes, plots will not look correct even though we inputed the exactly precise locations of gauges. One highly possible reason for that is our region is not resolved enough. Consequently, GeoClaw might recognize topography of locations wrongly. To solve this problem, we will use the AMRClaw algorithm to furthur refine regions. 

To elaborate, let's take Tampa Bay as an example. 

This is the Tampa Bay region when no refinement is added:
<div>
<img src="pic10.png" width="400"/>
</div>

But this is the Tampa Bay region when `minimum refinement level of 5`, `maximum refinement level of 6`, `refinement_ratios = [2, 2, 2, 3, 4, 4]` is added:
<div>
<img src="pic11.png" width="400"/>
</div>

It is very obvious that if we don't add refinement levels to this region, no gauges inside Tampa Bay can be recognized correctly since the program will assume they are on ''land''. 

To add specific regions to refine, simply add lines to `setrun.py` like these:

```
regions = rundata.regiondata.regions
regions.append([3, 5, clawdata.t0, clawdata.tfinal, -84, -82.5, 28, 30.5]) # Zoom-in Cedar Key
regions.append([3, 5, clawdata.t0, clawdata.tfinal, -84.47, -82.38, 29.4,30.17]) # Zoom-in Apalachee Bay
regions.append([5, 6, clawdata.t0, clawdata.tfinal, -82.89, -82.38, 27.49, 28.04]) # Zoom-in Tampa Bay
... (add more zoom-in regions)

```

Also, add refinement regions around gauges you select if necessary (some gauge may be in open water which we don't need to worry about, but some may be in narrow waterways that could not be recognized without a good refinement).

```
regions.append([4, 5, clawdata.t0, clawdata.tfinal, -83.1, -82.7, 27.85, 28.15]) # Clearwater Beach
... (add more regions of gauges)

```

<hr style="border:1.5px solid gray">

## Step 8: Execute Program

Finally we are ready to run the simulation. Make sure all files (setrun.py, setplot.py, makefile) are in the correct directory, then 

```
$ cd DirectoryPath
```

to the working directory. For this example, as set up by Step 1, it will be 

```
$ cd elsa
```

In that working directory, we can either use 
```
$ make all 

$ make .plots 
```
to execute. 

If errors are encountered, we can follow the sequence of 
```
$ make .exe

$ make .data

$ make .output

$ make .plots
```
to debug. To eliminate all executable files and restart, we can use 
```
$ make clean
```

Once the simulation runs successfully, we can see the result at `DirectoryPath/_plots/_PlotIndex.html`.
<div>
<img src="pic12.png" width="600"/>
</div>

<h4><center>Contact: jz3445@columbia.edu</center></h4>